In [ ]:
%cd ..

In [ ]:
import altair as alt
from altair_saver import save
import pandas as pd
from util import load_experiment

In [ ]:
baseline_df, val_df, test_df = load_experiment("position_bias_strength")

In [ ]:
test_df["position_bias"] = test_df.simulation_simulator_user_model_position_bias

In [ ]:
model2name = {
    "Neural PBM - Unbiased": "PBM - True bias",
    "Neural PBM - Biased": "PBM - Naive",
    "Neural PBM - Estimated bias": "PBM - Estimated bias",
    "Pointwise IPS - Unbiased": "Point. IPS - True bias",
    "Pointwise IPS - Biased": "Point. IPS / PBM - Naive",
}

test_df.model = test_df.model.map(model2name)
test_df = test_df[test_df.model != "Point. IPS / PBM - Naive"]

In [ ]:
chart = (alt.Chart(test_df, width=350, height=175, title="Istella-S").mark_line().encode(
    x=alt.X("position_bias", title="Position bias severity η"),
    y=alt.Y("mean(nDCG@10)", scale=alt.Scale(domain=[0.65, 0.75])),
    color=alt.Color("model", legend=None),
) + alt.Chart(test_df).mark_point().encode(
    x="position_bias",
    y=alt.Y("mean(nDCG@10)", scale=alt.Scale(domain=[0.65, 0.75])),
    color=alt.Color("model"),
    shape="model"
) + alt.Chart(test_df).mark_errorband(extent="ci").encode(
    x="position_bias",
    y=alt.Y("nDCG@10", scale=alt.Scale(domain=[0.65, 0.75])),
    color=alt.Color("model", legend=None),
)).resolve_scale(
    color="independent",
    shape="independent"
).configure_legend(
    orient="bottom",
    title=None,
    labelFont="serif",
    labelFontSize=16,
).configure_title(
    fontSize=16,
    fontWeight="normal",
    font="serif"
).configure_axis(
    titlePadding=5,
    titleFontSize=16,
    titleFontWeight="normal",
    titleFont="serif",
    labelFontSize=14,
    labelFontWeight="normal",
    labelFont="serif",
    tickCount=6
)

save(chart, "figures/position_bias_recovery.pdf")
chart